## LlamaIndex <> Langchain Integrations

This demo notebook shows how you can provide integrations between LlamaIndex and Langchain. It provides the following examples:
- Using LlamaIndex as a callable tool with a Langchain agent
- Using LlamaIndex as a memory module; this allows you to insert arbitrary amounts of conversation history with a Langchain chatbot!

In [1]:
from dotenv import find_dotenv, load_dotenv
import logging
import sys

load_dotenv(find_dotenv())
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Using LlamaIndex as a Callable Tool

In [8]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader(input_files=['/media/sunhuawei/data/Projects/playground/data/paul_graham_essay.txt']).load_data()
index = VectorStoreIndex.from_documents(documents=documents)

In [10]:
index.set_index_id('paul_graham_essay')
index.storage_context.persist(persist_dir='/media/sunhuawei/data/Projects/playground/data/storage/pg_essay')

In [11]:
tools = [
    Tool(
        name = "LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),
]

In [12]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0)  # gpt-3.5
agent_executor = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory)

In [13]:
agent_executor.run(input="hi, i am bob")

'Hello Bob! How can I assist you today?'

In [14]:
agent_executor.run(input="What did the author do growing up?")

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1455 tokens
> [get_response] Total LLM token usage: 1455 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


'\nThe author grew up writing short stories and programming on an IBM 1401. He also nagged his father to buy him a TRS-80 microcomputer, which he used to write simple games, a program to predict how high his model rockets would fly, and a word processor. He also studied philosophy in college, but eventually switched to AI.'

#### Using GPT Index as a memory module

In [15]:
# try using GPT List Index!
from langchain import OpenAI

from llama_index import ListIndex
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

In [16]:
index = ListIndex([])

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [17]:
# set Logging to DEBUG for more detailed outputs
# NOTE: you can also use a conversational chain

memory = GPTIndexChatMemory(
    index=index, 
    memory_key="chat_history", 
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True
)
llm=OpenAI(temperature=0)
agent_executor = initialize_agent([], llm, agent="conversational-react-description", memory=memory)

In [18]:
agent_executor.run(input="hi, i am bob")

INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 0 tokens
> [get_response] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens


'Hi Bob, nice to meet you! How can I help you today?'

In [19]:
# NOTE: the query now calls the ListIndex memory module. 
agent_executor.run(input="what's my name?")

INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 65 tokens
> [get_response] Total LLM token usage: 65 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


/home/sunhuawei/anaconda3/lib/python3.10/site-packages/llama_index/data_structs/node.py:182: UserWarning: .doc_id is deprecated, use .node.ref_doc_id instead
  warnings.warn(".doc_id is deprecated, use .node.ref_doc_id instead")


INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens


'Your name is Bob!'

In [22]:
index.ref_doc_info

{'0fce5c89-a34b-408f-847c-dd1d8a14c68a': RefDocInfo(doc_ids=['83493872-2532-41ce-b64d-bc4f29243e61'], extra_info={}),
 '43e6ef87-ad10-4801-bc7e-b1ef94169331': RefDocInfo(doc_ids=['87d0df48-3b98-4857-9219-2f7face60749'], extra_info={}),
 '854d77bb-5000-49b7-b7c3-f2f649c9f803': RefDocInfo(doc_ids=['ea30f15b-a95d-423d-8b89-6b387a3a9559'], extra_info={}),
 'c72ef093-2b7f-4e62-87e4-6b54db3efa8e': RefDocInfo(doc_ids=['c9ea7b4a-9970-419e-bb2b-1c6fdb2cd9a4'], extra_info={})}

In [25]:
index.storage_context.persist(persist_dir='/media/sunhuawei/data/Projects/playground/data/storage/chat_history')